In [1]:
# for each gene, for each developmental time point, search for the SectionDataSet 

from allensdk.api.queries.rma_api import RmaApi
import pandas as pd
api = RmaApi()


# user input directory name here for saving the files at the end
directory='D:\Data\DevelopingAllenMouseAPI-master\API data\GridData\\'

#ages = ['E11.5','E13.5','E15.5','E18.5','P4','P14','P28']
ages = ['E11.5']
age_str = "'" + "','".join(ages) + "'"

criteria=['[failed$eqfalse],'
          'products[abbreviation$eqDevMouse],'
          'specimen(donor(age[name$in%s])),' %age_str,
          'plane_of_section[name$eqsagittal]'
         ]
experiment=api.model_query('SectionDataSet',
                            criteria="".join(criteria),
                            include='probes(gene),specimen(donor(age))',
                            start_row=0,
                            num_rows='all')


In [2]:
# extract the info
expID_list=[]
age_list=[]
geneID_list=[]
geneAcronym_list=[]

for counter, value in enumerate(experiment):
    #only keep those experiments with entrez ID available
    if experiment[counter]['probes']:
        if not (experiment[counter]['probes'][0]['gene']['entrez_id']==None):
            expID_list.append(experiment[i]['id']) 
            age_list.append(experiment[i]['specimen']['donor']['age']['name'])
            geneID_list.append(experiment[i]['probes'][0]['gene']['entrez_id'])
            geneAcronym_list.append(experiment[i]['probes'][0]['gene']['acronym'])
    

In [3]:
# create the URL
API_PATH='http://api.brain-map.org/grid_data/download/'
#http://api.brain-map.org/grid_data/download/72109410?include=energy
URL_list=[]
dataType=['energy','intensity']
dataType_str = ','.join(dataType)

for j in expID_list: # for each experiment
    URL=(('%s' %(API_PATH)) +\
          '%d' % (j) +\
          '?include=' +\
          '%s' %(dataType_str)) 
    URL_list.append(URL)


In [4]:
import requests, zipfile, StringIO

for j in range(len(URL_list)): #len(URL_list)
    r = requests.get(URL_list[j], stream=True)
    z = zipfile.ZipFile(StringIO.StringIO(r.content))
    dirName=''.join([directory,age_list[j],'_',str(geneID_list[j])])
    z.extractall(dirName)

In [5]:
used = set()
age_unique = [x for x in age_list if x not in used and (used.add(x) or True)]
used = set()
geneID_unique = [x for x in geneID_list if x not in used and (used.add(x) or True)]
used = set()
geneAcronym_unique = [x for x in geneAcronym_list if x not in used and (used.add(x) or True)]
  

In [6]:
import csv

file_str=''.join([directory,'timePoints.csv'])
with open(file_str, 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(age_unique)
    
file_str=''.join([directory,'geneEntrez.csv'])
with open(file_str, 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(geneID_unique)
    
file_str=''.join([directory,'geneAbbreviations.csv'])
with open(file_str, 'wb') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(geneAcronym_unique)

In [ ]:
#for counter, value in enumerate(experiment):
    #if experiment[counter]['probes']:
        #if (experiment[counter]['probes'][0]['gene']['entrez_id']==None):
            #print('no gene entrez here')

#for item, location in ITEMS_IN_WORLD.iteritems():
#if (CURRENT_LOCATION==location):
#print ("Found!")  #Or whatever you intend to do

In [ ]:
#experiment_try=pd.DataFrame(experiment)
#experiment_try.to_csv('experiment_try.csv')